# From text to Graph


Este notebook es la mínima funcionalidad para convertir texto + ontología en grafo de conocimiento.

In [1]:
# PROCESAMIENTO DE DATOS
from langchain_docling import DoclingLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer

# AGENTES Y ORQUESTACIÓN
from langchain_ollama import ChatOllama, OllamaEmbeddings
from typing import List
from langgraph.graph import END, START, StateGraph


from IPython.display import Image, display, Markdown

c:\Users\alejandro.gl\Documents\_projects\Onto_Research\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.chains import GraphSparqlQAChain
from langchain_community.graphs import RdfGraph

graph = RdfGraph(
    source_file="https://spec.industrialontologies.org/ontology/supplychain/SupplyChain/",
    standard="rdf",
    local_copy="test.ttl",
)
Markdown(graph.get_schema)
llm = ChatOllama(model="llama3.2", temperature=0, base_url="http://localhost:11434")
embeddings = OllamaEmbeddings(model="llama3.2", base_url="http://localhost:11434")
sparql_chain = GraphSparqlQAChain.from_llm(
    llm, 
    graph=graph, 
    verbose=True,
    allow_dangerous_requests=True,
    return_sparql_query=True,
)
sparql_chain.invoke("Que es un producto?")

In [ ]:
def metadata_from_dataset(df: pd.DataFrame) -> dict:
    '''
    Extrae metadatos básicos del dataset: columnas, muestra de datos y tipo de dato.
    
    Args:
        df (pd.DataFrame): DataFrame a analizar
        
    Returns:
        dict: Diccionario con metadatos básicos de cada columna
    '''
    metadata = {}
    
    for column in df.columns:
        unique_values = df[column].unique()
        
        # Determinar si es único o múltiple
        if len(unique_values) == 1:
            uniqueness = "único"
            sample_data = unique_values[0]
        else:
            uniqueness = "múltiple"
            sample_data = unique_values[:3].tolist()  # Primeros 3 valores como muestra
        
        metadata[column] = {
            "tipo_dato": str(df[column].dtype),
            "valores_unicos": uniqueness,
            "muestra_datos": sample_data
        }
    
    return metadata

In [ ]:
dataset_metadata = metadata_from_dataset(df)

for k,v in dataset_metadata.items():
    print(k,v)
    #sparql_chain.invoke(f"Que es {k}?")